In [1]:
import os
from os.path import join
from io import StringIO
from glob import glob
from textwrap import dedent

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import QDYN
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [2]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-12-20_mg_method_comparison_dicke2_noise.json',
}
register_doit_as_IPython_magic()

In [3]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [4]:
! qdyn_version

QDYN 2.0dev revision dfd21683b898e790985b519daac35aeef7721f6c (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=intel, parallel-oct, backtraces, no-debug, no-no-ipo
  compiled with ifort on Mon Nov 13 14:54:23 2017 on host mlhpc2


$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Compare variants of Krotov's method for optimization of a two-node Dicke state

This does some extra calculations to get better statistics on the behavior of scaling of the noise

In [5]:
# We'll use different runfolders, but we want to re-use any data from the
# earlier calculations, which only did powers of two in the number of
# trajectories
! rsync -a ./data/method_comparison_dicke2/ ./data/method_comparison_dicke2_noise/

In [6]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    root = './data/method_comparison_dicke2_noise/'
    if frac_part == '00':
        T_str = "%s" % int_part
    else:
        T_str = '%s_%s' % (int_part, frac_part)
    rf = "T%s_%s_ntrajs%s" % (T_str, row['variant'], row['n_trajs'])
    return join(root, rf)

## action wrappers

In [7]:
from src.doit_actions_v2 import (
    run_traj_prop, update_config, wait_for_clusterjob,
    write_rho_prop_custom_config, collect_rho_prop_errors)

In [8]:
def submit_optimization(rf, n_trajs, variant, task):
    """Asynchronously run optimization, using either the
    standard QDYN implementation (variant = 'normal') or the 'crossoct' QDYN
    implementation at a particular revision (variant='cross')

    Does not support use of MPI, so this should only run on a workstation with
    more cores than the total number of trajectories
    """
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    bodies = {
        'normal': dedent(r'''
    {module_load}
    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS={threads} qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_ss .
    '''),
        'cross': dedent(r'''
    {module_load}
    module load prefix/crossoct

    qdyn_version | grep 66f6d24d2b07b9cc || {cancel_cmd}

    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS={threads} qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_cross .
    ''')}
    body = bodies[variant]
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    jobscript.n_trajs = str(int(n_trajs))
    jobscript.resources['ppn'] = 1 # not using MPI
    jobscript.resources['threads'] = min(int(n_trajs), 78)
    jobscript.threads = str(jobscript.resources['threads'])
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))


## custom uptodate routines

In [9]:
from src.qdyn_model_v2 import pulses_uptodate

## OCT task definitions

In [10]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    n_nodes = 2
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if rf in jobs:
            continue
        mcwf = {  #   variant => whether to use MCWF
            'rho': False, 'nonherm': False, 'independent': True,
            'cross': True}
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                # write the density matrix propagation data
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2,
                    mcwf=False, non_herm=False, # this defines rho-prop
                    config='config_rho')),
                # write the data for the optimization
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=mcwf[row['variant']],
                    non_herm=(row['variant'] != 'rho'),
                    lambda_a=row['lambda_a'], keep_pulses='all',
                    iter_stop=int(row['iter_stop']), J_T_conv=row['J_T_conv'],
                    config='config'))],
            'targets': [join(rf, 'config_rho'), join(rf, 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [11]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row
    (adjusting lambda_a, iter_stop and J_T_conv only)"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']),
                    J_T_conv=row['J_T_conv']))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [12]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    optimization_variant = {
        'rho': 'normal',
        'nonherm': 'normal',
        'independent': 'normal',
        'cross': 'cross',
    }
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ],
                 dict(n_trajs=row['n_trajs'],
                      variant=optimization_variant[row['variant']]))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [13]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

Additional tasks for evaluating the optimization success through a propagation
in Liouville space are defined farther below

##  OCT Submission

In [14]:
params_data_str = r'''
# T  lambda_a      variant  n_trajs   iter_stop   J_T_conv
  5     0.001  independent        1        5000       1e-8
  5     0.001  independent        2        5000       1e-8
  5     0.001  independent        3        5000       1e-8
  5     0.001  independent        4        5000       1e-8
  5     0.001  independent        5        5000       1e-8
  5     0.001  independent        8        5000       1e-8
  5     0.001  independent       10        5000       1e-8
  5     0.001  independent       15        5000       1e-8
  5     0.001  independent       20        5000       1e-8
  5     0.001  independent       25        5000       1e-8
  5     0.001  independent       30        5000       1e-8
  5     0.001  independent       32        5000       1e-8
  5     0.001  independent       35        5000       1e-8
  5     0.001  independent       40        5000       1e-8
  5     0.001  independent       45        5000       1e-8
  5     0.001  independent       50        5000       1e-8
  5     0.001  independent       55        5000       1e-8
  5     0.001  independent       60        5000       1e-8
  5     0.001  independent       64        5000       1e-8
  5     0.001  independent       70        5000       1e-8
  5     0.001  independent       75        5000       1e-8
  5     0.001  independent       80        5000       1e-8
  5     0.001  independent       85        5000       1e-8
  5     0.001  independent       90        5000       1e-8
  5     0.001  independent      100        5000       1e-8
  5     0.001  independent      110        5000       1e-8
  5     0.001  independent      120        5000       1e-8
  5     0.001  independent      128        5000       1e-8
  5     0.001  cross              2        5000       1e-8
  5     0.001  cross              3        5000       1e-8
  5     0.001  cross              4        5000       1e-8
  5     0.001  cross              5        5000       1e-8
  5     0.001  cross              8        5000       1e-8
  5     0.001  cross             10        5000       1e-8
  5     0.001  cross             15        5000       1e-8
  5     0.001  cross             20        5000       1e-8
  5     0.001  cross             25        5000       1e-8
  5     0.001  cross             30        5000       1e-8
  5     0.001  cross             32        5000       1e-8
  5     0.001  cross             35        5000       1e-8
  5     0.001  cross             40        5000       1e-8
  5     0.001  cross             45        5000       1e-8
  5     0.001  cross             50        5000       1e-8
  5     0.001  cross             55        5000       1e-8
  5     0.001  cross             60        5000       1e-8
  5     0.001  cross             64        5000       1e-8
  5     0.001  cross             70        5000       1e-8
  5     0.001  cross             75        5000       1e-8
  5     0.001  cross             80        5000       1e-8
  5     0.001  cross             85        5000       1e-8
  5     0.001  cross             90        5000       1e-8
  5     0.001  cross            100        5000       1e-8
  5     0.001  cross            110        5000       1e-8
  5     0.001  cross            120        5000       1e-8
  5     0.001  cross            128        5000       1e-8
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['T', 'lambda_a', 'variant', 'n_trajs', 'iter_stop', 'J_T_conv'],
        converters={
            'T': float, 'lambda_a': float, 'variant': str,
            'n_trajs': int, 'iter_stop': int, 'J_T_conv': float})

In [15]:
print(params_df.to_string())

      T  lambda_a      variant  n_trajs  iter_stop      J_T_conv
0   5.0     0.001  independent        1       5000  1.000000e-08
1   5.0     0.001  independent        2       5000  1.000000e-08
2   5.0     0.001  independent        3       5000  1.000000e-08
3   5.0     0.001  independent        4       5000  1.000000e-08
4   5.0     0.001  independent        5       5000  1.000000e-08
5   5.0     0.001  independent        8       5000  1.000000e-08
6   5.0     0.001  independent       10       5000  1.000000e-08
7   5.0     0.001  independent       15       5000  1.000000e-08
8   5.0     0.001  independent       20       5000  1.000000e-08
9   5.0     0.001  independent       25       5000  1.000000e-08
10  5.0     0.001  independent       30       5000  1.000000e-08
11  5.0     0.001  independent       32       5000  1.000000e-08
12  5.0     0.001  independent       35       5000  1.000000e-08
13  5.0     0.001  independent       40       5000  1.000000e-08
14  5.0     0.001  indepe

In [16]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG, filename='./data/method_comparison_dicke2_noise.log')

In [17]:
%doit -n 40 wait_for_optimization

-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs1
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs2
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs3
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs4
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs5
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs8
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs10
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs15
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs20
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs25
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs30
-- create_runfolder:./data/method_comparison_dicke2_noise/T5_independent_ntrajs32
-- create_runfolder:./

In [18]:
x = 1